# IAU - Inteligentná analýza údajov (2024/2025)

#### Autori: Jan Lenhart (50 %), Marek Čederle (50 %)
##### Cvičenie: Pondelok 15:00, Cvičiaci: Ing. Oleksandr Lytvyn

<font color='salmon'>
    <b>Poznámka:</b>
    Fáza 1 - boli zachované nejaké časti, funkcie a knižnice, ktoré nám boli užitočné pri fáze 2, pri prezeraní fázy 2 je vhodné si fázu 1 zbaliť
</font>

## Fáza 1 - Prieskumná analýza

Na začiatok si importujeme knižnice a načítame dáta, do dátových rámcov (dataframov).

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from copy import copy
from scipy import stats
import statsmodels.api as sm

In [ ]:
pd.set_option("display.width", 2**20)
pd.set_option("display.max_columns", 32)

In [ ]:
df_connections = pd.read_csv('112/connections.csv', delimiter=',')
df_processes   = pd.read_csv('112/processes.csv', delimiter=',')

IMEI je unikátny identifikátor mobilného zariadenia, ktorý sa nachádza vo všetkých rámcoch, takže ho bude možné použiť ako primárny kľúč na spájanie tabuliek, ak to bude neskôr potrebné.

In [ ]:
def print_normality_test(df):
    if df.count() > 5000:
        stat, p_value = stats.kstest(df, 'norm')
        print(f"kstest for count={df.count()}: [stat: {stat}, p: {p_value}, norm: {p_value > 0.05}]")
    else:
        stat, p_value = stats.shapiro(df)
        print(f"stat: for count={df.count()}: [stat: {stat}, p: {p_value}, norm: {p_value > 0.05}]")

In [ ]:
print_normality_test(df_connections["c.katana"])
print_normality_test(df_connections["c.android.gm"])
print_normality_test(df_connections["c.android.chrome"])
print_normality_test(df_connections["c.dogalize"])

Počet záznamov je väčší ako 5000, preto bol použitý Kolmogorov-Smirnov (`kstest`) namiesto Shapiro-Wilk z ktorého vyplíva, že dáta vo vybraných stĺpcoch nespĺňajú normálovú distribúciu pretože p-hodnota je menšia ako 0.05.

In [ ]:
print_normality_test(df_processes["p.android.gm"])
print_normality_test(df_processes["p.system"])
print_normality_test(df_processes["p.android.chrome"])
print_normality_test(df_processes["p.browser.provider"])
print_normality_test(df_processes["p.android.documentsui"])
print_normality_test(df_processes["p.android.packageinstaller"])


In [ ]:
target_predictor = 'mwra'
strong_predictors = pd.DataFrame()
moderate_predictors = pd.DataFrame()
weak_predictors = pd.DataFrame()

In [ ]:
def numerical_prediction(df):
    global strong_predictors, moderate_predictors, weak_predictors
    numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns
    corr_matrix = df[numerical_columns].corr()
    corrs = corr_matrix[[target_predictor]].sort_values(by=target_predictor, ascending=False)
    corrs = corrs.drop('mwra')
    sns.heatmap(corrs, annot=True, cmap='coolwarm')
    plt.title(f'Correlation of {target_predictor} with numerical predictors')
    plt.show()
    strong_predictors = pd.concat([strong_predictors, pd.DataFrame(np.where(abs(corrs) > 0.7, corrs, np.nan), columns=corrs.columns, index=corrs.index).dropna()])
    moderate_predictors = pd.concat([moderate_predictors, pd.DataFrame(np.where((abs(corrs) > 0.5) & (abs(corrs) <= 0.7), corrs, np.nan), columns=corrs.columns, index=corrs.index).dropna()])
    weak_predictors = pd.concat([weak_predictors, pd.DataFrame(np.where((abs(corrs) > 0.25) & (abs(corrs) <= 0.5), corrs, np.nan), columns=corrs.columns, index=corrs.index).dropna()])

Funkcia `numerical_prediction` získa prediktory pre dátový rámec, ktorý dostane ako parameter. Získané prediktory akumuluje do globálnych premenných `strong_predictors`, `moderate_predictors` a `weak_predictors`. To sú kategorizované premenné, ktoré v páre so stĺpcom `mwra` majú koreláciu viac ako `0.7` pre `strong`, viac ako `0.5` a menej ako `0.7` pre `moderate` a viac ako `0.25` (bolo by lepšie, keby toto číslo bolo 0.3, ale chceli sme zvýšiť počet prediktorov a ak budú dávať zlé výsledky, môžeme ich kedykoľvek vyhodiť) a menej ako `0.5` pre `weak`. Taktiež, pri volaní zobrazí heatmapu vypočítaných korelácií, aby poskytla vizuálny prehľad o vzťahoch medzi premennými.

In [ ]:
numerical_prediction(df_connections)

Tento graf nám hovorí o tom, ktoré premenné z dátového rámca `df_connections` korelujú s našou predikovanou premennou `mwra`.
Môžeme vidieť že stĺpce `c.dogalize`, `c.android.chrome` a `c.katana` majú najvyššiu koreláciu s predikovanou premennou `mwra`.

In [ ]:
numerical_prediction(df_processes)

Tento graf nám hovorí o tom, ktoré premenné z dátového rámca `df_processes` korelujú s našou predikovanou premennou `mwra`.
Môžeme vidieť že stĺpce `p.android.gm`, `p.system`, `p.android.chrome`, `p.android.documentsui` a `p.android.packageinstaller` majú najvyššiu koreláciu s predikovanou premennou `mwra`.

In [ ]:
print(strong_predictors)

In [ ]:
print(moderate_predictors)

In [ ]:
print(weak_predictors)

Tu môžeme vidieť že nemáme žiadne silné predikátory (korelácia `> 0.7`).
Máme však stredne silné a slabšie predikátory.

#### 1.1.E Zamyslenie k riešeniu projektu

Pomocou párovej analýzi sme zistili nejakú koreláciu resp. závisloť premenných. Konkrétne mali najsilnejšiu koreláciu s predikovanou premennou `mwra` tieto stĺpce:
- c.dogalize
- p.android.gm
- p.android.packageinstaller

Kedže sa nachádzajú v iných dátových rámcoch, bude potrebné ich spojiť pomocou `imei`.

Podľa toho, ako dáta vyzerajú a čo sme zistili, tak si myslíme že ide o nejaké využitie CPU s tým že CPU môže mať viacero jadier a preto hodnoty premenných nedávajú súčet 100 ale viacej čo by znamenalo že by mohlo ísť o viacej jadrové CPU (napr. 8), ale toto je iba naša teória.

Je vysoko pravdepodobné že bude treba spájať dané dáta pretože aj `devices.csv` aj `processes.csv` majú stĺpec `mwra` a budeme muset zistit koreláciu naprieč týmito dátami.

### 1.2 Identifikácia problémov, integrácia a čistenie dát

#### 1.2.A Identifikácia a prvotné riešenie problémov v dátach

In [ ]:
connections_string_columns = set(copy(df_connections.select_dtypes(include=['object']).columns))
processes_string_columns = set(copy(df_processes.select_dtypes(include=['object']).columns))
string_columns = list(connections_string_columns | processes_string_columns)
print(string_columns)

##### Vymazanie duplikátov v dátach

In [ ]:
def remove_duplicates(df):
    pre_count = df.duplicated().count()
    df = df.drop_duplicates()
    post_count = df.duplicated().count()
    print(f"removed {pre_count - post_count} duplicates")
    return df

In [ ]:
df_connections = remove_duplicates(df_connections)

In [ ]:
df_processes = remove_duplicates(df_processes)

#### 1.2.B Chýbajúce hodnoty (missing values)

In [ ]:
def printNonesAndNA(label, df):
    nulls = df.isnull().sum()
    if (nulls.sum()):
        df_temp = pd.DataFrame(np.where(nulls > 0, nulls, np.nan), index=nulls.index, columns=['null_count']).dropna()
        df_temp['percentage'] = df_temp.apply(lambda x: x / df.shape[0] * 100)
        print(label)
        print(df_temp)
        print()
        return df_temp

In [ ]:
printNonesAndNA("df_connections", df_connections)
printNonesAndNA("df_processes", df_processes)

#### 1.2.C Vychýlené hodnoty (outlier detection)

Na ukážku si vykreslíme boxploty pre vybrané atribúty a zistíme, či sa v nich nachádzajú nejaké vychýlené hodnoty. Naše atribúty sú premenné, ktoré najviac korelovali s predikovanou premennou `mwra` a teda sú to:
- c.dogalize
- p.android.gm
- p.android.packageinstaller

In [ ]:
sns.boxplot(x='mwra', y='c.dogalize', data=df_connections)

In [ ]:
sns.boxplot(x='mwra', y='p.android.gm', data=df_processes)

In [ ]:
sns.boxplot(x='mwra', y='p.android.packageinstaller', data=df_processes)

Následne identifikujeme outlierov pomocou IQR metódy a overíme že ich skutočne máme.

In [ ]:
# zdroj z cvicenia
def identify_outliers(data):
    lower = data.quantile(0.25) - 1.5 * stats.iqr(data)
    upper = data.quantile(0.75) + 1.5 * stats.iqr(data)
    return data[(data > upper) | (data < lower)]

In [ ]:
def identify_outliers_for_all_columns(df):
    val = {}
    columns = df.select_dtypes(exclude=['object']).columns
    for column in columns:
        outlier_count = identify_outliers(df[column]).count()
        if outlier_count > 0:
            val[column] = outlier_count
    return val

In [ ]:
identify_outliers_for_all_columns(df_connections)

In [ ]:
identify_outliers_for_all_columns(df_processes)

Zistili sme že ich máme relatívne málo vzhľadom na počet záznamov, takže ich môžeme odstrániť.

In [ ]:
def remove_outliers_for_all_columns(df):
    df_temp = df.copy()
    indices = pd.DataFrame({})
    accumulated_indices = set()
    for column in df_temp.select_dtypes(exclude=['object']).columns:
        accumulated_indices |= set(identify_outliers(df_temp[column]).index)
    df_temp = df_temp.drop(accumulated_indices)
    return df_temp

In [ ]:
df_connections = remove_outliers_for_all_columns(df_connections)
df_processes = remove_outliers_for_all_columns(df_processes)

Ostránime iba outlierov z dátového rámca `df_processes` a `df_connections` pretože, v dátovom rámci `df_devices` a `df_profiles` nemáme žiadne numerické hodnoty, s ktorými je vhodné pracovať. (Zemepisné súradnice a unikátne identifikátory)
Po odstránení outlierov si možeme znova identifikovať či náhodou nejakých ešte nemáme.

In [ ]:
identify_outliers_for_all_columns(df_connections)

In [ ]:
identify_outliers_for_all_columns(df_processes)

Zistili sme že napriek úvodnému odstráneniu mám stále outlierov. Deje sa to z toho dôvodu, že sa vlastne distibúcia posunie a preto sa nám objavili nový outliery. Keby takto pokračujeme a odstraňujeme stále dáta, tak by sme prišli o veľký počet záznamov a preto sme sa rozhodli ich nahradiť hraničnými hodnotami (5% a 95%).

In [ ]:
sns.boxplot(x='mwra', y='p.browser.provider', data=df_processes)

In [ ]:
sns.histplot(df_processes['p.browser.provider'], kde=True, color='blue')

In [ ]:
def replace_outliers_with_percentiles(df):
    df_temp = df.copy()
    for column in df_temp.select_dtypes(exclude=['object']).columns:
        lower_bound = df_temp[column].quantile(0.05)
        upper_bound = df_temp[column].quantile(0.95)
        lower_outliers = df_temp[column] < lower_bound
        upper_outliers = df_temp[column] > upper_bound
        df_temp.loc[lower_outliers, column] = lower_bound
        df_temp.loc[upper_outliers, column] = upper_bound
    return df_temp

In [ ]:
df_connections = replace_outliers_with_percentiles(df_connections)
df_processes = replace_outliers_with_percentiles(df_processes)

In [ ]:
identify_outliers_for_all_columns(df_connections)

In [ ]:
identify_outliers_for_all_columns(df_processes)

In [ ]:
# histogram
sns.histplot(df_processes['p.browser.provider'], kde=True, color='blue')

Ako vidíme z výpisu dát, tak sa nám podarilo všetkých outlierov bud odstrániť alebo nahradiť hraničnými hodnotami.
Jedniou výnimkou je `p.browser.provider` a to z dôvodu ako vyzerá jeho distribúcia. Preto sme sa rozhodli to nemeniť, pretože by sme pri iteratívnom odstraňovaní prišli o veľký počet záznamov.

### 1.3 Formulácia a štatistické overenie hypotéz o dátach

#### 1.3.A Formulácia hypotéz

##### Hypotéza č.1

Zvolíme si náš "significance level" na $\alpha = 0.05$. (95%)

Null hypothesis (nulová hypotéza):

$H_0$: Premenná `p.android.gm` má v priemere rovnakú váhu v stave malware-related-activity ako v normálnom stave.

Alternative hypothesis (alternatívna hypotéza):
$H_1$ = $H_A$: Premenná `p.android.gm` má v priemere inú váhu v stave malware-related-activity ako v normálnom stave. (Nižšiu alebo vyššiu)

**Najskôr si musíme overiť či dáta spĺňajú normálovú distribúciu**

Takto vyzerá náš boxplot, ktorý nám ukazuje distribúciu hodnôt pre premennú `p.android.gm` v závislosti od `mwra`.

In [ ]:
sns.boxplot(x='mwra', y='p.android.gm', data=df_processes)

In [ ]:
p_android_gm_0 = df_processes.loc[df_processes.mwra == 0, 'p.android.gm']
p_android_gm_0.describe()

In [ ]:
sns.histplot(p_android_gm_0)

In [ ]:
p_android_gm_1 = df_processes.loc[df_processes.mwra == 1, 'p.android.gm']
p_android_gm_1.describe()

In [ ]:
sns.histplot(p_android_gm_1)

Z týchto distribúcií môžeme vidieť, že nevyzerajú ako normálne distribúcie, preto musíme pokračovať s overením.

In [ ]:
p_android_gm_0_outliers = identify_outliers(p_android_gm_0)
p_android_gm_1_outliers = identify_outliers(p_android_gm_1)

In [ ]:
p_android_gm_0_outliers.count()

In [ ]:
p_android_gm_1_outliers.count()

In [ ]:
p_android_gm_0 = p_android_gm_0.drop(p_android_gm_0_outliers.index)
p_android_gm_1 = p_android_gm_1.drop(p_android_gm_1_outliers.index)

In [ ]:
sns.histplot(p_android_gm_1)

Vidíme že aj po vyhodení outlierov sa nám distribúcia nezmenila a preto môžeme pokračovať s testovaním.

In [ ]:
_ = sm.ProbPlot(p_android_gm_0, fit=True).qqplot(line='45')

In [ ]:
_ = sm.ProbPlot(p_android_gm_1, fit=True).qqplot(line='45')


Ani `QQ-plot` nám nepotvrdil normálnu distribúciu. Pretože aby bola distribúcia normálna, tak by sa body museli nachádzať na priamke definovanej ako $x=y$.

In [ ]:
p_android_gm_0.count()

In [ ]:
p_android_gm_1.count()

Musíme použiť štatistický test, ktorý nám povie či dáta majú normálnu distribúciu alebo nie. Použijeme `Kolmogorov-Smirnov` test namiesto `Shapiro-Wilk` pretože počet záznamov je väčší ako 5000.

In [ ]:
print_normality_test(p_android_gm_0)

In [ ]:
print_normality_test(p_android_gm_1)

Výsledok testu nám potvrdil, že dáta nemajú normálnu distribúciu, preto musíme použiť neparametrický test na overenie hypotézy. Pretože p hodnota je menšia ako 0.05.

Použijeme `Mann-Whitney U Test` pretože máme práve dve premenné a zároveň spĺňajú predpoklady pre tento test (vzorka musí byť aspoň 20).

In [ ]:
stats.mannwhitneyu(p_android_gm_0, p_android_gm_1)

Keďže nám vyšlo p menšie ako 0.05 tak zamietame nulovú hypotézu a prijímame alternatívnu hypotézu, že premenná `p.android.gm` má v priemere inú váhu v stave malware-related-activity ako v normálnom stave.

##### Hypotéza č.2

Zvolíme si náš "significance level" na $\alpha = 0.05$. (95%)

Null hypothesis (nulová hypotéza):

$H_0$: Premenná `p.android.packageinstaller` má v priemere rovnakú váhu v stave malware-related-activity ako v normálnom stave.

Alternative hypothesis (alternatívna hypotéza):
$H_1$ = $H_A$: Premenná `p.android.packageinstaller` má v priemere inú váhu v stave malware-related-activity ako v normálnom stave. (Nižšiu alebo vyššiu)

**Najskôr si musíme overiť či dáta spĺňajú normálovú distribúciu**

Takto vyzerá náš boxplot, ktorý nám ukazuje distribúciu hodnôt pre premennú `p.android.packageinstaller` v závislosti od `mwra`.

In [ ]:
sns.boxplot(x='mwra', y='p.android.packageinstaller', data=df_processes)

In [ ]:
p_android_packageinstaller_0 = df_processes.loc[df_processes.mwra == 0, 'p.android.packageinstaller']
p_android_packageinstaller_0.describe()

In [ ]:
sns.histplot(p_android_packageinstaller_0)

In [ ]:
p_android_packageinstaller_1 = df_processes.loc[df_processes.mwra == 1, 'p.android.packageinstaller']
p_android_packageinstaller_1.describe()

In [ ]:
sns.histplot(p_android_packageinstaller_1)

Z týchto distribúcií môžeme vidieť, že nevyzerajú ako normálne distribúcie, preto musíme pokračovať s overením.

In [ ]:
p_android_packageinstaller_0_outliers = identify_outliers(p_android_packageinstaller_0)
p_android_packageinstaller_1_outliers = identify_outliers(p_android_packageinstaller_1)

In [ ]:
p_android_packageinstaller_0_outliers.count()

In [ ]:
p_android_packageinstaller_1_outliers.count()

In [ ]:
p_android_packageinstaller_0 = p_android_packageinstaller_0.drop(p_android_packageinstaller_0_outliers.index)
p_android_packageinstaller_1 = p_android_packageinstaller_1.drop(p_android_packageinstaller_1_outliers.index)

In [ ]:
sns.histplot(p_android_packageinstaller_0)

In [ ]:
sns.histplot(p_android_packageinstaller_1)

Vidíme že aj po vyhodení outlierov sa nám distribúcia nezmenila a preto môžeme pokračovať s testovaním.

In [ ]:
_ = sm.ProbPlot(p_android_packageinstaller_0, fit=True).qqplot(line='45')

In [ ]:
_ = sm.ProbPlot(p_android_packageinstaller_1, fit=True).qqplot(line='45')

Ani `QQ-plot` nám nepotvrdil normálnu distribúciu. Pretože aby bola distribúcia normálna, tak by sa body museli nachádzať na priamke definovanej ako $x=y$.

In [ ]:
p_android_packageinstaller_0.count()

In [ ]:
p_android_packageinstaller_1.count()

Musíme použiť štatistický test, ktorý nám povie či dáta majú normálnu distribúciu alebo nie. Použijeme `Shapiro-Wilk` test pre jednu premennú a `Kolmogorov-Smirnov` pre druhú premennú pretože pri jednej je počet záznamov je nižší ako 5000 a pri druhej vyšší.

In [ ]:
# Shapiro-Wilk test
print_normality_test(p_android_packageinstaller_0)

In [ ]:
# Kolmogorov-Smirnov test
print_normality_test(p_android_packageinstaller_1)

Výsledok testu nám potvrdil, že dáta nemajú normálnu distribúciu, preto musíme použiť neparametrický test na overenie hypotézy. Pretože p hodnota je menšia ako 0.05.

Použijeme `Mann-Whitney U Test` pretože máme práve dve premenné a zároveň spĺňajú predpoklady pre tento test (vzorka musí byť aspoň 20).

In [ ]:
stats.mannwhitneyu(p_android_packageinstaller_0, p_android_packageinstaller_1)

Keďže nám vyšlo p menšie ako 0.05 tak zamietame nulovú hypotézu a prijímame alternatívnu hypotézu, že premenná `p.android.packageinstaller` má v priemere inú váhu v stave malware-related-activity ako v normálnom stave.

#### 1.3.B Overenie štatistickej sily

Kedže nám p values vyšli všetky menšie ako 0.001, tak môžeme povedať že naše testy mali veľkú štatistickú silu.

## Fáza 2 - Predspracovanie údajov

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import f_regression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
df_connections = pd.read_csv('112/connections.csv', delimiter=',')
df_processes   = pd.read_csv('112/processes.csv', delimiter=',')

### 2.1 Realizácia predspracovania dát

#### 2.1.A Dáta si rozdeľte na trénovaciu a testovaciu množinu podľa vami preddefinovaného pomeru. Ďalej pracujte len s trénovacím datasetom.

In [ ]:
merged_data = df_connections.merge(df_processes, on=['imei', 'ts'], how='inner')

In [ ]:
merged_data['ts'] = pd.to_numeric(pd.to_datetime(merged_data['ts'], errors='coerce'))

Po načítaní datasetov z `.csv` súborov sme si ich spojili na základe spoločného stĺpca `imei` a `ts` a upravili sme dátový typ stĺpca `ts` aby sa nám s ním neskôr dalo pracovať.

Na základe výsledkov a zistení vo fázy 1, sme použili iba datasety `processes.csv` a `connections.csv` pretože obsahujú numerické hodnoty, ktoré sú vhodné pre modelovanie.

In [ ]:
merged_data.describe()

In [ ]:
merged_data.info()

In [ ]:
merged_data.loc[merged_data['mwra_x'] != merged_data['mwra_y']]['imei'].count() / merged_data['imei'].count()

In [ ]:
merged_data = merged_data.drop_duplicates()

In [ ]:
merged_data = merged_data.drop(columns=['mwra_y'])

In [ ]:
merged_data = merged_data.rename(columns={"mwra_x": "mwra"})

In [ ]:
X = merged_data.drop(columns='mwra')
y = merged_data['mwra']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Objavili sa nám 2 stĺpce `mwra`, tak smeoverili, či sú medyi nimi nejaké rozdiely, konkrétne `mwra_x` a `mwra_y` a zistili sme že sú rovnaké (bol to dôsledok joinu, pretože `mwra` sa nachádzala v oboch súboroch), preto sme jeden z nich odstránili.
Následne sme si rozdelili naše dáta na trénovaciu a testovaciu časť podľa pomeru 80:20 (trénovacia : testovacia).

#### 2.1.B Transformujte dáta na vhodný formát pre ML t.j. jedno pozorovanie musí byť opísané jedným riadkom a každý atribút musí byť v numerickom formáte (encoding). zIteratívne integrujte aj kroky v predspracovaní dát z prvej fázy (missing values, outlier detection) ako celok. 

##### Missing values (Chýbajúce hodnoty)

In [ ]:
X_train.isnull().sum().sum()

In [ ]:
imputer = SimpleImputer(strategy='mean')
X_train_impute = imputer.fit_transform(X_train)
X_train = pd.DataFrame(X_train_impute, columns=X_train.columns, index=X_train.index)

Zistili sme, že nemáme žiadne missing values v našich dátach, avšak ako prípravu na pipeline sme si urobili `imputer`, ktorý by prípadné chýbajúce hodnoty nahradil za priemer (keďže nepracujeme s testovacím datasetom ale iba trénovacím).

Keďže všetky naše hodnoty majú numerický formát, tak nemusíme robiť encoding.

##### Handling outliers (Ošetrovanie outlierov)

In [ ]:
def handle_outliers(X, y):
    X_temp = X.copy()
    indices_remove = set()
    for column in X.columns:
        outliers = identify_outliers(X[column])
        if outliers.count() / X[column].count() < 0.05:
            indices_remove.update(outliers.index)
        
    X_temp = X.drop(index=indices_remove)
    y = y.drop(index=indices_remove)

    X_temp = replace_outliers_with_percentiles(X_temp)
    
    return X_temp, y

In [ ]:
X_train.count().max()

In [ ]:
sns.histplot(X_train['p.browser.provider'], kde=True)

In [ ]:
identify_outliers_for_all_columns(X_train)

In [ ]:
X_train, y_train = handle_outliers(X_train, y_train)

In [ ]:
X_train.count().max()

In [ ]:
identify_outliers_for_all_columns(X_train)

In [ ]:
sns.histplot(X_train['p.browser.provider'], kde=True)

Outlierov sme vyriešili podobne ako v prvej fáze. Ak bol počet outlierov v jednom stĺpci vacčší ako 5% z celkového počtu záznamov, tak sme ich nahradili hraničnými hodnotami (5 a 95 percentil), ak boli menšie, tak sme ich odstránili. Na ukážku sme zobrazili graf premennej `p.browser.provider` a zistili sme že aj po odstránení outlierov, máme zase ďalších, ale ďaľej sme nepokračovali v ich odstraňovaní, pretože by sme prišli o veľký počet záznamov.

#### 2.1.C Transformujte atribúty dát pre strojové učenie podľa dostupných techník minimálne: scaling (2 techniky), transformers (2 techniky) a ďalšie. Cieľom je aby ste testovali efekty a vhodne kombinovali v dátovom pipeline (od časti 2.3 a v 3. fáze).

##### Ukážka distribúcií našich dát pred škálovaním a transformáciou

In [ ]:
moderate_predictors

In [ ]:
weak_predictors

In [ ]:
def plot_predictor_distribution(in_df, series_mwra, predictors):
    df = in_df.copy()
    df['mwra'] = series_mwra
    num_predictors = len(predictors)
    fig, axes = plt.subplots(1, num_predictors, figsize=(5 * num_predictors, 5), sharey=True)  # sharey=True to share the y-axis scale
    for i, predictor in enumerate(predictors.index):
       sns.kdeplot(data=df, x=predictor, hue="mwra", fill=True, common_norm=False, alpha=0.5, ax=axes[i])
    fig.tight_layout()
    plt.show()

In [ ]:
plot_predictor_distribution(merged_data, merged_data['mwra'], moderate_predictors)

In [ ]:
plot_predictor_distribution(merged_data, merged_data['mwra'], weak_predictors)

##### Škálovanie (Scaling)

Vyskúšame viacej metód škálovania, a neskôr zistíme ktorá je pre nás najvhodnejšia v spojení s transformáciou.

In [ ]:
min_max_scaler = MinMaxScaler()
X_train_normalized = min_max_scaler.fit_transform(X_train)
X_train_normalized = pd.DataFrame(X_train_normalized, columns=X_train.columns, index=X_train.index)

standard_scaler = StandardScaler()
X_train_standardized = standard_scaler.fit_transform(X_train)
X_train_standardized = pd.DataFrame(X_train_standardized, columns=X_train.columns, index=X_train.index)

robust_scaler = RobustScaler()
X_train_robust = robust_scaler.fit_transform(X_train)
X_train_robust = pd.DataFrame(X_train_robust, columns=X_train.columns, index=X_train.index)

Na ukážku používame naše predikátory z fázy 1, pretože predpokladáme že to budú najdôležitejšie premenné resp. črty (features).

**Normalizácia dát (MinMaxScaler)**

In [ ]:
plot_predictor_distribution(X_train_normalized, y_train, moderate_predictors)

In [ ]:
plot_predictor_distribution(X_train_normalized, y_train, weak_predictors)

**Štandardizácia dát (StandardScaler)**

In [ ]:
plot_predictor_distribution(X_train_standardized, y_train, moderate_predictors)

In [ ]:
plot_predictor_distribution(X_train_standardized, y_train, weak_predictors)

**Robustné škálovanie (RobustScaler)**

In [ ]:
plot_predictor_distribution(X_train_robust, y_train, moderate_predictors)

In [ ]:
plot_predictor_distribution(X_train_robust, y_train, weak_predictors)

Grafy nám všake ukazujú distribúcie, ktoré by mohli byť podobné normálnej distribúcii (s nejakými výnimkami), avšak sú "skewed" a ich konce (tails) sú viacej špičaté, čo bolo spôsobné nahradením outlierov hraničnými hodnotami.

##### Transformations

Vyskúšame taktiež viacej metód transformovania pre každé škálovanie, a neskôr zistíme ktorá je pre nás najvhodnejšia.

In [ ]:
power_transformer = PowerTransformer(method='yeo-johnson')
quantile_transformer = QuantileTransformer(n_quantiles=1000, random_state=0)

In [ ]:
X_train_normalized_power = power_transformer.fit_transform(X_train_normalized)
X_train_normalized_quantile = quantile_transformer.fit_transform(X_train_normalized)

X_train_standardized_power = power_transformer.fit_transform(X_train_standardized)
X_train_standardized_quantile = quantile_transformer.fit_transform(X_train_standardized)

X_train_robust_power = power_transformer.fit_transform(X_train_robust)
X_train_robust_quantile = quantile_transformer.fit_transform(X_train_robust)

In [ ]:
X_train_normalized_power = pd.DataFrame(X_train_normalized_power, columns=X_train_normalized.columns, index=X_train_normalized.index)
X_train_normalized_quantile = pd.DataFrame(X_train_normalized_quantile, columns=X_train_normalized.columns, index=X_train_normalized.index)
X_train_standardized_power = pd.DataFrame(X_train_standardized_power, columns=X_train_standardized.columns, index=X_train_standardized.index)
X_train_standardized_quantile = pd.DataFrame(X_train_standardized_quantile, columns=X_train_standardized.columns, index=X_train_standardized.index)
X_train_robust_power = pd.DataFrame(X_train_robust_power, columns=X_train_robust.columns, index=X_train_robust.index)
X_train_robust_quantile = pd.DataFrame(X_train_robust_quantile, columns=X_train_robust.columns, index=X_train_robust.index)

<font color='#38C570'>
    <b>
        Transformácia dát (PowerTransformer), s normalizáciou dát (MinMaxScaler)
    </b>
</font>

In [ ]:
plot_predictor_distribution(X_train_normalized_power, y_train, moderate_predictors)

In [ ]:
plot_predictor_distribution(X_train_normalized_power, y_train, weak_predictors)

**Transformácia dát (QuantileTransformer), s normalizáciou dát (MinMaxScaler)**

In [ ]:
plot_predictor_distribution(X_train_normalized_quantile, y_train, moderate_predictors)

In [ ]:
plot_predictor_distribution(X_train_normalized_quantile, y_train, weak_predictors)

**Transformácia dát (PowerTransformer), s štandardizáciou dát (StandardScaler)**

In [ ]:
plot_predictor_distribution(X_train_standardized_power, y_train, moderate_predictors)

In [ ]:
plot_predictor_distribution(X_train_standardized_power, y_train, weak_predictors)

**Transformácia dát (QuantileTransformer), s štandardizáciou dát (StandardScaler)**

In [ ]:
plot_predictor_distribution(X_train_standardized_quantile, y_train, moderate_predictors)

In [ ]:
plot_predictor_distribution(X_train_standardized_quantile, y_train, weak_predictors)

**Transformácia dát (PowerTransformer), s robustným škálovaním (RobustScaler)**

In [ ]:
plot_predictor_distribution(X_train_robust_power, y_train, moderate_predictors)

In [ ]:
plot_predictor_distribution(X_train_robust_power, y_train, weak_predictors)

**Transformácia dát (QuantileTransformer), s robustným škálovaním (RobustScaler)**

In [ ]:
plot_predictor_distribution(X_train_robust_quantile, y_train, moderate_predictors)

In [ ]:
plot_predictor_distribution(X_train_robust_quantile, y_train, weak_predictors)

#### 2.1.D Zdôvodnite Vaše voľby/rozhodnutie pre realizáciu (t.j. zdokumentovanie)

In [ ]:
X_train = X_train_normalized_power.copy()

Vyskúšali sme si viacej kombinácií škálovania a transformácií. Rozhodli sme sa, že ďalej (neskôr do pipeline) použijeme `MinMaxScaler` na škálovanie a `PowerTransformer` na transformovanie dát. Je to z dôvodu, že `MinMaxScaler` škáluje dáta na jednotný rozsah (v našom prípade <0; 1>). Používa sa, keď nemáme žiadnych outlierov alebo už boli odstránené, čo je presne náš prípad. Síce máme stále nejakých outlierov v premennej `p.browser.provider`, ale tá neni našim predikátorom, takže by to nemalo ovplyvňovať naše výsledky. `PowerTransformer` sme si vybrali pretože vie transofrmovať dáta približne na normálnu distribúciu (blízko normálnej distribúcii), čo je vhodné pre niektoré ML (Machine Learning) modely, pretože to môže prinášať lepšie výsledky.

### 2.2 Výber atribútov pre strojové učenie

#### 2.2.A Zistite, ktoré atribúty (features) vo vašich dátach pre ML sú informatívne k predikovanej premennej (minimálne 3 techniky s porovnaním medzi sebou). 

Vyskúšame 3 metódy na výber čŕt, ktoré nám pomôžu zistiť, ktoré črty (features) sú najinformatívnejšie pre predikciu našej predikovanej premennej `mwra`.

##### Mutual Information

In [ ]:
mutual_info_regression_selector = SelectKBest(mutual_info_regression, k=5)
X_train_featured_mutual_info_regression = mutual_info_regression_selector.fit_transform(X_train, y_train)
X_train_featured_mutual_info_regression = pd.DataFrame(X_train_featured_mutual_info_regression, columns=X_train.columns[mutual_info_regression_selector.get_support()], index=X_train.index)

In [ ]:
X_train_featured_mutual_info_regression.head(0)

##### F-value

In [ ]:
f_regression_selector = SelectKBest(f_regression, k=5)
X_train_featured_f_regression = f_regression_selector.fit_transform(X_train, y_train)
X_train_featured_f_regression = pd.DataFrame(X_train_featured_f_regression, columns=X_train.columns[f_regression_selector.get_support()], index=X_train.index)

In [ ]:
X_train_featured_f_regression.head(0)

##### Linear SVC(Support Vector Classifier)

In [ ]:
lsvc_selector = SelectFromModel(LinearSVC(C=0.0001, penalty="l1", dual=False))
X_train_featured_lsvc = lsvc_selector.fit_transform(X_train, y_train)
X_train_featured_lsvc = pd.DataFrame(X_train_featured_lsvc, columns=X_train.columns[lsvc_selector.get_support()], index=X_train.index)

In [ ]:
X_train_featured_lsvc.head(0)

#### 2.2.B Zoraďte zistené atribúty v poradí podľa dôležitosti. 

Taktiež vieme pomocou `Mutual Information` urobiť ranking našich čŕt (features) podľa dôležitosti.

In [ ]:
mutual_info_scores = mutual_info_regression_selector.scores_

feature_scores = pd.DataFrame({
    'Feature': X_train.columns,
    'Mutual Information Score': mutual_info_scores
})

feature_scores = feature_scores.sort_values(by='Mutual Information Score', ascending=False).reset_index(drop=True)

plt.figure(figsize=(10, 8))
sns.barplot(x='Mutual Information Score', y='Feature', data=feature_scores)
plt.title("Features Ranked by Mutual Information Score")
plt.xlabel("Mutual Information Score")
plt.ylabel("Features")
plt.show()


#### 2.2.C Zdôvodnite Vaše voľby/rozhodnutie pre realizáciu (t.j. zdokumentovanie)

Vyskúšali sme 3 metódy výberu čŕt (feature selection), a zisili sme, že nám dávajú totožné výsledky v zmysle nájdených "silných" čŕt (features).


Medzi ne patrí:
- c.dogalize
- p.system
- p.android.gm
- p.android.packageinstaller
- p.browser.provider

Tieto čŕty sú najinformatívnejšie pre predikovanie našej predikovanej premennej `mwra`.

V našom pipeline sme sa rozhodli, že použijeme metódu "Mutual Information" (`mutual_info_regression`) pre výber čŕt.

Metóda "Mutual information" meria závislosť medzi jednotlivými črtami a cieľovou (predikovanou) premennou bez toho, aby predpokladala konkrétny typ vzťahu medzi nimi (lineárny alebo nelineárny). Keďže sa jej výsledky aj zhodovali s ostatnými metódami, tak sme sa ju práve z tohto dôvodu rozhodli použiť v našom pipeline.

### 2.3 Replikovateľnosť predspracovania

#### 2.3.A Upravte váš kód realizujúci predspracovanie trénovacej množiny tak, aby ho bolo možné bez ďalších úprav znovu použiť na predspracovanie testovacej množiny v kontexte strojového učenia.

##### Zrekapitulovanie toho, čo sme urobili manuálne bez "sklearn pipeline"

```py

df_connections = pd.read_csv('112/connections.csv', delimiter=',')
df_processes   = pd.read_csv('112/processes.csv', delimiter=',')

merged_data = df_connections.merge(df_processes, on=['imei', 'ts'], how='inner')
merged_data['ts'] = pd.to_numeric(pd.to_datetime(merged_data['ts'], errors='coerce'))
merged_data = merged_data.drop_duplicates()
merged_data = merged_data.drop(columns=['mwra_y'])
merged_data = merged_data.rename(columns={"mwra_x": "mwra"})

X = merged_data.drop(columns='mwra')
y = merged_data['mwra']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

imputer = SimpleImputer(strategy='mean')
X_train_impute = imputer.fit_transform(X_train)
X_train = pd.DataFrame(X_train_impute, columns=X_train.columns, index=X_train.index)

X_train, y_train = handle_outliers(X_train, y_train)

min_max_scaler = MinMaxScaler()
X_train_normalized = min_max_scaler.fit_transform(X_train)
X_train_normalized = pd.DataFrame(X_train_normalized, columns=X_train.columns, index=X_train.index)

power_transformer = PowerTransformer(method='yeo-johnson')
X_train_normalized_power = power_transformer.fit_transform(X_train_normalized)
X_train_normalized_power = pd.DataFrame(X_train_normalized_power, columns=X_train_normalized.columns, index=X_train_normalized.index)

X_train = X_train_normalized_power.copy()

mutual_info_regression_selector = SelectKBest(mutual_info_regression, k=5)
X_train_featured_mutual_info_regression = mutual_info_regression_selector.fit_transform(X_train, y_train)
X_train_featured_mutual_info_regression = pd.DataFrame(X_train_featured_mutual_info_regression, columns=X_train.columns[mutual_info_regression_selector.get_support()], index=X_train.index)

X_train = X_train_featured_mutual_info_regression.copy()
```

##### Vytvorenie pipeline

Vytvorenie vlastnej triedy na ošetrenie outlierov.

In [ ]:
# co-engineered with ChatGPT
class OutlierHandler(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=1.5, outlier_fraction=0.05):
        self.threshold = threshold
        self.outlier_fraction = outlier_fraction
        self.imputer = SimpleImputer(strategy='median')

    def fit(self, X, y=None):
        self.iqr = np.percentile(X, 75, axis=0) - np.percentile(X, 25, axis=0)
        self.lower_bound = np.percentile(X, 25, axis=0) - self.threshold * self.iqr
        self.upper_bound = np.percentile(X, 75, axis=0) + self.threshold * self.iqr
        return self

    def transform(self, X):
        outliers_mask = np.any((X < self.lower_bound) | (X > self.upper_bound), axis=1)
        num_outliers = np.sum(outliers_mask)
        total_samples = X.shape[0]
        outlier_fraction = num_outliers / total_samples
        
        if outlier_fraction < self.outlier_fraction:
            return X[~outliers_mask]
        else:
            X_imputed = X.copy()
            X_imputed[outliers_mask] = np.nan
            return self.imputer.fit_transform(X_imputed)
        # X_imputed = X.copy()
        # X_imputed[outliers_mask] = np.nan
        # return self.imputer.fit_transform(X_imputed)

Trieda `OutlierHandler` je akokeby vlastný transformer a vie transformovať vstupné dáta tak, že malé množstvo outlierov odstráni a veľké množstvo outlierov nahradí hraničnými hodnotami. Slúžia na to funkcie `fit` a `transform`.

- `fit()` - zistuje či sú záznamy obsahujú outlierov
- `transform()` - odstraňuje alebo nahradzuje outlierov

#### 2.3.B Využite možnosti sklearn.pipeline

Prevedenie celého nášho doterajšieho manuálneho postupu do "sklearn pipeline".

In [ ]:
df_connections = pd.read_csv('112/connections.csv', delimiter=',')
df_processes = pd.read_csv('112/processes.csv', delimiter=',')

merged_data = df_connections.merge(df_processes, on=['imei', 'ts'], how='inner')
merged_data['ts'] = pd.to_numeric(pd.to_datetime(merged_data['ts'], errors='coerce'))
merged_data = merged_data.drop_duplicates()
merged_data = merged_data.drop(columns=['mwra_y'])
merged_data = merged_data.rename(columns={"mwra_x": "mwra"})

X = merged_data.drop(columns='mwra')
y = merged_data['mwra']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
preprocessor_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('outlier_handler', OutlierHandler(threshold=1.5, outlier_fraction=0.05)),
    ('scaler', MinMaxScaler()),
    ('power_transformer', PowerTransformer(method='yeo-johnson')),
    ('feature_selection', SelectKBest(mutual_info_regression, k=5))],
    verbose=True
)

preprocessor_pipeline.fit(X_train, y_train)

Náš pipeline funguje nasledovne:
- `SimpleImputer` - nahradí chýbajúce hodnoty za priemer
- `OutlierHandler` - odstráni alebo nahradí outlierov
- `MinMaxScaler` - škáluje dáta na jednotný rozsah
- `PowerTransformer` - pokúsi sa transoformovať dáta na normálnu distribúciu (resp. distribúcie blízkej normálnej distribúcii)
- `SelectKBest` - vyberie najinformatívnejšie črty (features) pre predikciu predikovanej premennej (`mwra`)

In [ ]:
preprocessed_train = preprocessor_pipeline.transform(X_train)
preprocessed_train = pd.DataFrame(preprocessed_train, columns=X_train.columns[preprocessor_pipeline.named_steps['feature_selection'].get_support()], index=X_train.index)

preprocessed_test = preprocessor_pipeline.transform(X_test)
preprocessed_test = pd.DataFrame(preprocessed_test, columns=X_test.columns[preprocessor_pipeline.named_steps['feature_selection'].get_support()], index=X_test.index)

Ukážka trénovacích a testovacích dát po prejdení cez pipeline.

In [ ]:
preprocessed_train.head()

In [ ]:
preprocessed_test.head()

Uloženie preprocesovaných dát do `.csv` súborov.

In [ ]:
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))

In [ ]:
# pre_preprocess_train = X_train.copy()
# pre_preprocess_train = pre_preprocess_train[['c.dogalize']]
# pre_preprocess_train = pre_preprocess_train.drop(columns=['c.dogalize'])
# pre_preprocess_train["mwra"] = y_train
# pre_preprocess_train.to_csv('pre_preprocess_train.csv')
# pre_preprocess_test = X_test.copy()
# pre_preprocess_test = pre_preprocess_test[['c.dogalize']]
# pre_preprocess_test = pre_preprocess_test.drop(columns=['c.dogalize'])
# pre_preprocess_test["mwra"] = y_test
# pre_preprocess_test.to_csv('pre_preprocess_test.csv')

In [ ]:
# preprocessed_train = preprocessed_train[['c.dogalize']]
# preprocessed_train = preprocessed_train.drop(columns=['c.dogalize'])

preprocessed_train["mwra"] = y_train
preprocessed_train.to_csv('preprocessed_train.csv', index=False)

In [ ]:
# preprocessed_test = preprocessed_test[['c.dogalize']]
# preprocessed_test = preprocessed_test.drop(columns=['c.dogalize'])

preprocessed_test["mwra"] = y_test
preprocessed_test.to_csv('preprocessed_test.csv', index=False)

## Fáza 3 - Strojové učenie

<font color='salmon'>
    <b>Poznámka:</b>
    Toto je iba začiatok práce na fáze 3.
</font>

In [ ]:
classifier_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor_pipeline),
    ('classifier', DecisionTreeClassifier()),
])

In [ ]:
classifier_pipeline.fit(X_train, y_train)

In [ ]:
y_pred = classifier_pipeline.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8411371237458194

Classification Report:
               precision    recall  f1-score   support

         0.0       0.84      0.71      0.77      1116
         1.0       0.84      0.92      0.88      1874

    accuracy                           0.84      2990
   macro avg       0.84      0.81      0.82      2990
weighted avg       0.84      0.84      0.84      2990



## Zdroje

**Fáza 1**

Prednášky a cvičenia z predmetu IAU.

[IAU Github repozitár](https://github.com/FIIT-IAU/IAU-course)

[Scipy dokumentácia](https://docs.scipy.org/doc/scipy/reference/index.html)

[Numpy dokumentácia](https://numpy.org/doc/)

[Pandas dokumentácia](https://pandas.pydata.org/docs/)

[Typical Analysis Procedure](https://work.thaslwanter.at/Stats/html/statsAnalysis.html)

**Fáza 2**

Predošlé zdroje plus:

[Scikit-learn dokumentácia](https://scikit-learn.org/1.5/api/index.html)

[Feature importance](https://machinelearningmastery.com/calculate-feature-importance-with-python/)

[Scaling](https://machinelearningmastery.com/standardscaler-and-minmaxscaler-transforms-in-python/)

[Transformations](https://machinelearningmastery.com/power-transforms-with-scikit-learn/)